<a href="https://colab.research.google.com/github/rosehelfrich/Ky_School_data/blob/main/Predict_Classification%2C_and_Highly_Impacted_Schools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Column / Row options
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

# Two decimal places and the thousands separator
pd.options.display.float_format = '{:,.2f}'.format

# KY df - load and clean

In [4]:
# Load the data frames
df_scores = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/df_scores.csv')
ky_spending_df = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/preprocessed_df.csv')

# Select only the necessary columns from ky_spending_df
ky_spending_df = ky_spending_df.loc[:,['End Year', 'School Code', 'Level', 'Reported Spending per student', 'Money Difference per school',
       'Money Difference per student', 'Estimated Spending per student']]

# Create a new column 'Rating' in df_scores
df_scores['Rating'] = df_scores['Classification']

# Reorder the columns of df_scores
reordered_columns = ['End Year', 'District Code', 'District', 'School Code', 'School', 'Level', 
                     'Proficiency Rate', 'Classification', 'Rating']
df_scores = df_scores[reordered_columns] 


In [5]:
# Merge into one df
df = pd.merge(df_scores, ky_spending_df, on=['End Year', 'School Code', 'Level'], how='left')


# Sort df & Reset index
df.dropna(subset=['End Year'], inplace=True)
df.sort_values(by=['End Year', 'District', 'School Code'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
0,"2,012.00",1,Adair County,1010,Adair County High School,HS,55.40,Needs Improvement,Needs Improvement,"6,457.00","-36,414.00",-52.00,"6,405.00"
1,"2,012.00",1,Adair County,1014,Adair County Middle School,MS,48.30,Needs Improvement,Needs Improvement,"6,719.00","-55,680.00",-144.00,"6,575.00"
2,"2,012.00",1,Adair County,1016,Adair County Elementary School,ES,62.90,Proficient,Proficient,"7,261.00","-33,813.00",-57.00,"7,204.00"
3,"2,012.00",5,Allen County,5010,Allen County Primary Center,ES,49.70,Needs Improvement,Needs Improvement,"7,927.00","190,512.00",210.00,"8,137.00"
4,"2,012.00",5,Allen County,5015,James E Bazzell Middle School,MS,56.70,Needs Improvement,Needs Improvement,"7,644.00","78,624.00",178.00,"7,822.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,"2,022.00",601,Woodford County,601050,Southside Elementary School,ES,75.40,High,High,NaN,NaN,NaN,NaN
10918,"2,022.00",601,Woodford County,601075,Simmons Elementary School,ES,69.70,Medium,Medium,NaN,NaN,NaN,NaN
10919,"2,022.00",601,Woodford County,601084,Woodford County High School,HS,67.70,High,High,NaN,NaN,NaN,NaN
10920,"2,022.00",601,Woodford County,601090,Huntertown Elementary School,ES,77.00,High,High,NaN,NaN,NaN,NaN


In [7]:
df.nunique()


End Year                             9
District Code                      172
District                           172
School Code                       1145
School                            1121
Level                                3
Proficiency Rate                   748
Classification                      11
Rating                              11
Reported Spending per student     5840
Money Difference per school       7888
Money Difference per student      1124
Estimated Spending per student    5904
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        10922 non-null  float64
 1   District Code                   10922 non-null  int64  
 2   District                        10922 non-null  object 
 3   School Code                     10922 non-null  int64  
 4   School                          10922 non-null  object 
 5   Level                           10922 non-null  object 
 6   Proficiency Rate                10922 non-null  float64
 7   Classification                  8471 non-null   object 
 8   Rating                          8471 non-null   object 
 9   Reported Spending per student   8603 non-null   float64
 10  Money Difference per school     8626 non-null   float64
 11  Money Difference per student    8626 non-null   float64
 12  Estimated Spending per student  

In [9]:
# Take out "/Progressing"
df.replace(['Distinguished/Progressing', 'Proficient/Progressing', 'Needs Improvement/Progressing'], 
           ['Distinguished', 'Proficient', 'Needs Improvement'], inplace=True)

df['Rating'].replace(['Distinguished', 'Proficient', 'Needs Improvement'], np.NaN,inplace=True)

df = df.round({'Proficiency Rate': 0, 'Reported Spending per student': -1, 'Money Difference per school': -2, 
                               'Money Difference per student':-1, 'Estimated Spending per student': 1})

In [10]:
# This df now keeps the original data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        10922 non-null  float64
 1   District Code                   10922 non-null  int64  
 2   District                        10922 non-null  object 
 3   School Code                     10922 non-null  int64  
 4   School                          10922 non-null  object 
 5   Level                           10922 non-null  object 
 6   Proficiency Rate                10922 non-null  float64
 7   Classification                  8471 non-null   object 
 8   Rating                          2513 non-null   object 
 9   Reported Spending per student   8603 non-null   float64
 10  Money Difference per school     8626 non-null   float64
 11  Money Difference per student    8626 non-null   float64
 12  Estimated Spending per student  

In [11]:
df.nunique()

End Year                             9
District Code                      172
District                           172
School Code                       1145
School                            1121
Level                                3
Proficiency Rate                    99
Classification                       8
Rating                               5
Reported Spending per student     1471
Money Difference per school       3252
Money Difference per student       182
Estimated Spending per student    5904
dtype: int64

In [12]:
df['Classification'].value_counts(dropna=False)

Needs Improvement    3059
NaN                  2451
Proficient           1505
Distinguished        1394
Medium                815
Low                   751
High                  528
Very High             213
Very Low              206
Name: Classification, dtype: int64

In [13]:
df['Level'].value_counts(dropna=False)

ES    6188
MS    2738
HS    1996
Name: Level, dtype: int64

In [14]:
df[df['School'].str.contains("Newcomer")]

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student


# Predict classification 

In [15]:
# from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [16]:
predict_df = df.copy()

# Rationalization: The top and bottom data has the least room for error. 
predict_df['Classification'].replace(['Very High', 'High', 'Medium', 'Low', 'Very Low'],
                     ['Distinguished', np.NaN, np.NaN, np.NaN,'Needs Improvement'], inplace=True)


In [17]:
predict_df['Classification'].value_counts(dropna=False)

NaN                  4545
Needs Improvement    3265
Distinguished        1607
Proficient           1505
Name: Classification, dtype: int64

In [18]:
predict_df['Classification'].value_counts(dropna=False)

NaN                  4545
Needs Improvement    3265
Distinguished        1607
Proficient           1505
Name: Classification, dtype: int64

In [19]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        10922 non-null  float64
 1   District Code                   10922 non-null  int64  
 2   District                        10922 non-null  object 
 3   School Code                     10922 non-null  int64  
 4   School                          10922 non-null  object 
 5   Level                           10922 non-null  object 
 6   Proficiency Rate                10922 non-null  float64
 7   Classification                  6377 non-null   object 
 8   Rating                          2513 non-null   object 
 9   Reported Spending per student   8603 non-null   float64
 10  Money Difference per school     8626 non-null   float64
 11  Money Difference per student    8626 non-null   float64
 12  Estimated Spending per student  

In [20]:
def fill_nulls_w_avg(column):
  avg_by_school_code = predict_df.groupby(['School Code']).mean()
  rows_to_update = predict_df[predict_df[column].isnull()]

  for index in rows_to_update.index:
    school_code = rows_to_update['School Code'][index]
    avg = avg_by_school_code.loc[school_code, column]
    predict_df.loc[index, column] = avg

# Use function to fill in select columns with the average school information. 
fill_nulls_w_avg('Reported Spending per student')
fill_nulls_w_avg('Money Difference per school')
fill_nulls_w_avg('Money Difference per student')
fill_nulls_w_avg('Estimated Spending per student')

In [21]:
# Convert objects to floats

predict_df['Classification'].replace(['Distinguished', 'Proficient', 'Needs Improvement'], [0, 1, 2], inplace=True)

predict_df['Level'].replace(['ES', 'MS', 'HS'], [0, 1, 2], inplace=True)

## Shuffle, Balance, Split, Scale

In [22]:
unscaled_df = predict_df.loc[:,['End Year', 'Proficiency Rate', 'Reported Spending per student', 
             'Money Difference per school', 'Level', 'Classification']]

unscaled_df.dropna(axis =0, inplace=True)
unscaled_df.reset_index(drop=True, inplace=True)

#Shuffle df
unscaled_df = unscaled_df.sample(frac=1, random_state=15)

In [23]:
# Balance the data
balanced_df = unscaled_df.iloc[0:0]

for i in unscaled_df['Classification'].unique():  
  min_target_size = unscaled_df['Classification'].value_counts().min()
  target_values = unscaled_df[unscaled_df['Classification']==i] 
  fraction = round(min_target_size / target_values.shape[0], 2)
  sample_data = target_values.sample(frac=fraction, random_state=8)
  balanced_df = pd.concat([balanced_df, sample_data])

# Shuffle the data
balanced_df = balanced_df.sample(frac=1, random_state=3)

In [24]:
balanced_df['Classification'].value_counts(dropna=False)

0.00    1508
1.00    1503
2.00    1496
Name: Classification, dtype: int64

In [25]:
unscaled_inputs = balanced_df.values[:,:-1]

In [26]:
targets = balanced_df.values[:,-1]

In [27]:
# Split into training, validation, and test sets

unscaled_X_train, unscaled_X_set, y_train, y_set = train_test_split(unscaled_inputs, targets, test_size=0.2, random_state=15)
unscaled_X_valid, unscaled_X_test, y_valid, y_test = train_test_split(unscaled_X_set, y_set, test_size=0.5, random_state=13)

In [28]:
# Scale data
epsb_scaler = StandardScaler()

# Calculate and store the mean and sd
epsb_scaler.fit(unscaled_X_train)

# Apply the scaler
scaled_X_train = epsb_scaler.transform(unscaled_X_train)
scaled_X_valid = epsb_scaler.transform(unscaled_X_valid)
scaled_X_test = epsb_scaler.transform(unscaled_X_test)

In [29]:
scaled_X_train.shape

(3605, 5)

## Model

In [30]:
# Create the model
# Ran different rounds of the below model.  Some of the options I put in comments to the right.

input_size = 5
output_size = 3
hidden_layer_size = 15 # for this problem, optimal results were between 10-20 layers

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),    # attempted to make this linear, only use the proficency in the input layers, and it only came to 77% valid accuracy or less
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation='softmax') # the sigmoid function made the model decrease in accuracy
    ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [31]:
# Fit the model

batch_size = 50  # started with batch size 25
max_epochs = 100 # started with 20 epochs

#early_stopping = tf.keras.callbacks.EarlyStopping(patience=5) 

model.fit(scaled_X_train, 
          y_train, 
          batch_size=batch_size,
          epochs = max_epochs,
          #callbacks = [early_stopping],
          validation_data = (scaled_X_valid, y_valid),
          verbose =2)

# Ran rounds using a multi-index of School Code & End Year.  But didn't get above a 78% validation accuracy

Epoch 1/100
73/73 - 1s - loss: 1.0265 - accuracy: 0.4779 - val_loss: 0.9085 - val_accuracy: 0.6186 - 1s/epoch - 20ms/step
Epoch 2/100
73/73 - 0s - loss: 0.7696 - accuracy: 0.6641 - val_loss: 0.7144 - val_accuracy: 0.6851 - 209ms/epoch - 3ms/step
Epoch 3/100
73/73 - 0s - loss: 0.6182 - accuracy: 0.7259 - val_loss: 0.5891 - val_accuracy: 0.7118 - 182ms/epoch - 2ms/step
Epoch 4/100
73/73 - 0s - loss: 0.5258 - accuracy: 0.7767 - val_loss: 0.5118 - val_accuracy: 0.7472 - 176ms/epoch - 2ms/step
Epoch 5/100
73/73 - 0s - loss: 0.4695 - accuracy: 0.8033 - val_loss: 0.4702 - val_accuracy: 0.7783 - 235ms/epoch - 3ms/step
Epoch 6/100
73/73 - 0s - loss: 0.4341 - accuracy: 0.8202 - val_loss: 0.4447 - val_accuracy: 0.7894 - 185ms/epoch - 3ms/step
Epoch 7/100
73/73 - 0s - loss: 0.4122 - accuracy: 0.8291 - val_loss: 0.4284 - val_accuracy: 0.7938 - 202ms/epoch - 3ms/step
Epoch 8/100
73/73 - 0s - loss: 0.3980 - accuracy: 0.8325 - val_loss: 0.4195 - val_accuracy: 0.7960 - 242ms/epoch - 3ms/step
Epoch 9/10

In [32]:
# test the model

test_loss, test_accuracy = model.evaluate(scaled_X_test, y_test)

15/15 [==============================] - 0s 5ms/step - loss: 0.1603 - accuracy: 0.9335


In [33]:
# Print Pretty

print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.16. Test accuracy: 93.35%


In [34]:
# Save model
# Test accuracy for the saved model is 94%
#tf.keras.models.save_model(model, '/content/drive/MyDrive/Colab Data/KY EPSB/model', include_optimizer=True)

## Predict Classification using model

In [35]:
# Grab the data that we want to predict
inputs = predict_df.loc[:,['End Year', 'Proficiency Rate', 'Reported Spending per student', 
             'Money Difference per school', 'Level']]

# Convert to np array
# scale data
inputs = epsb_scaler.transform(inputs.values)


In [36]:
inputs.shape

(10922, 5)

In [37]:
pred_classification = model.predict(inputs)

342/342 [==============================] - 1s 2ms/step


In [38]:
# Convert to a df
pred_classification = pd.DataFrame(pred_classification, columns=[0, 1, 2])

In [39]:
# Select the column with the highest percentage
pred_classification = pred_classification.idxmax(axis=1)

In [40]:
pred_classification.shape

(10922,)

In [41]:
predict_df.index

RangeIndex(start=0, stop=10922, step=1)

In [42]:
predict_df['Predicted Classification'] = pred_classification

In [43]:
predict_df['Classification'].value_counts(dropna=False)


NaN     4545
2.00    3265
0.00    1607
1.00    1505
Name: Classification, dtype: int64

In [44]:
# Determine the accuracy of the predict_df using the test accuracy, and subtracting the values 
# we already know are inaccurate.
# end cnt is about 217

cnt = 0

for i in predict_df.index:
  if predict_df['Classification'][i] != predict_df['Predicted Classification'][i]:
    cnt += 1

model_accuracy = test_accuracy + (cnt / len(predict_df))

model_accuracy


1.388068228790861

In [45]:
predict_df

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student,Predicted Classification
0,"2,012.00",1,Adair County,1010,Adair County High School,2,55.00,2.00,NaN,"6,460.00","-36,400.00",-50.00,"6,405.00",2.00
1,"2,012.00",1,Adair County,1014,Adair County Middle School,1,48.00,2.00,NaN,"6,720.00","-55,700.00",-140.00,"6,575.00",2.00
2,"2,012.00",1,Adair County,1016,Adair County Elementary School,0,63.00,1.00,NaN,"7,260.00","-33,800.00",-60.00,"7,204.00",1.00
3,"2,012.00",5,Allen County,5010,Allen County Primary Center,0,50.00,2.00,NaN,"7,930.00","190,500.00",210.00,"8,137.00",2.00
4,"2,012.00",5,Allen County,5015,James E Bazzell Middle School,1,57.00,2.00,NaN,"7,640.00","78,600.00",180.00,"7,822.00",2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,"2,022.00",601,Woodford County,601050,Southside Elementary School,0,75.00,NaN,High,"10,725.00","70,762.50",136.25,"10,861.75",0.00
10918,"2,022.00",601,Woodford County,601075,Simmons Elementary School,0,70.00,NaN,Medium,"12,065.00","-14,650.00",-30.00,"12,033.38",2.00
10919,"2,022.00",601,Woodford County,601084,Woodford County High School,2,68.00,NaN,High,"9,492.50","28,962.50",23.75,"9,515.38",2.00
10920,"2,022.00",601,Woodford County,601090,Huntertown Elementary School,0,77.00,NaN,High,"10,847.50","115,625.00",247.50,"11,094.25",0.00


In [46]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        10922 non-null  float64
 1   District Code                   10922 non-null  int64  
 2   District                        10922 non-null  object 
 3   School Code                     10922 non-null  int64  
 4   School                          10922 non-null  object 
 5   Level                           10922 non-null  int64  
 6   Proficiency Rate                10922 non-null  float64
 7   Classification                  6377 non-null   float64
 8   Rating                          2513 non-null   object 
 9   Reported Spending per student   10876 non-null  float64
 10  Money Difference per school     10876 non-null  float64
 11  Money Difference per student    10876 non-null  float64
 12  Estimated Spending per student  

In [47]:
#predict_df.to_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/predict_df.csv', index = False)

# Nested Model

In [48]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        10922 non-null  float64
 1   District Code                   10922 non-null  int64  
 2   District                        10922 non-null  object 
 3   School Code                     10922 non-null  int64  
 4   School                          10922 non-null  object 
 5   Level                           10922 non-null  int64  
 6   Proficiency Rate                10922 non-null  float64
 7   Classification                  6377 non-null   float64
 8   Rating                          2513 non-null   object 
 9   Reported Spending per student   10876 non-null  float64
 10  Money Difference per school     10876 non-null  float64
 11  Money Difference per student    10876 non-null  float64
 12  Estimated Spending per student  

## Shuffle, Split, Scale

In [49]:
nested_df = predict_df.loc[:,['End Year', 'Level', 'Reported Spending per student', 'Money Difference per school',
                              'Proficiency Rate',  'Predicted Classification', 'Rating']]
nested_df['Rating'].replace(['Very High', 'High', 'Medium', 'Low', 'Very Low'], [0,1,2,3,4], inplace=True)
nested_df.drop_duplicates(inplace=True)
nested_df.dropna(axis =0, inplace=True)

#Shuffle data
nested_df = nested_df.sample(frac=1, random_state=7)
nested_df.reset_index(drop=True, inplace=True)

# Balanced the data. 
# Dilemma: Don't balance the dataset (81.95) OR balance based on Classification column OR balance based on Rating column 
# Decision: Run both and see which model does best. 

In [50]:
nested_df

,End Year,Level,Reported Spending per student,Money Difference per school,Proficiency Rate,Predicted Classification,Rating
0,"2,022.00",1,"9,967.50","-25,037.50",54.00,2.00,3.00
1,"2,022.00",0,"13,906.25","-92,312.50",36.00,2.00,3.00
2,"2,022.00",0,"9,155.00","-75,516.67",76.00,0.00,1.00
3,"2,019.00",1,"13,230.00","38,400.00",80.00,0.00,1.00
4,"2,022.00",0,"11,515.00","5,750.00",61.00,2.00,2.00
...,...,...,...,...,...,...,...
2480,"2,019.00",1,"11,160.00","-4,200.00",67.00,2.00,3.00
2481,"2,022.00",1,"10,778.75","-7,300.00",54.00,2.00,3.00
2482,"2,019.00",0,"11,200.00","25,200.00",75.00,0.00,2.00
2483,"2,019.00",0,"11,880.00","-76,100.00",91.00,0.00,0.00


In [51]:
nested_df.columns

Index(['End Year', 'Level', 'Reported Spending per student',
       'Money Difference per school', 'Proficiency Rate',
       'Predicted Classification', 'Rating'],
      dtype='object')

In [52]:
# Searched for information on unique and duplicated rows. 

test_df = nested_df.loc[:,['End Year', 'Level', 'Proficiency Rate', 'Rating']] 
test_df.drop_duplicates(inplace=True) 
test_df.sort_values(by=['Proficiency Rate'], inplace=True, ignore_index=True)

# Analyzed data.  The following is the number of unique rows with the columns included.
# 403 rows with 5 columns: 'End Year', 'Level','Proficiency Rate', 'Classification', 'Rating'
# 192 rows with 4 columns: 'Level','Proficiency Rate', 'Classification', 'Rating'
# 381 rows with 4 columns: 'End Year', 'Level', 'Proficiency Rate', 'Rating'
# 167 rows with 2 columns: 'Proficiency Rate', 'Rating'
# 107 rows with only year 2022 data and 2 columns: 'Proficiency Rate', 'Rating'
# 193 rows with only year 2022 data and 3 columns: 'Level','Proficiency Rate', 'Rating'

# The elementary, middle and high schools have various ways of measuring their Ratings according to proficency rate. 
# However even within these school levels, the rating is not only based on the proficency rate. 

In [53]:
# Conclusion from above analysis:
# A common assumption would be that the proficency rate, level, or year corresponds to the Rating.  
# By the analyzes done from above, this assumption is not valid.  The different groups kept yielding duplicated values. 

test_df.groupby(by=['Level', 'Proficiency Rate']).count().sort_values(by=['Rating'])

End Year  Rating
Level Proficiency Rate                  
0     10.00                    1       1
1     84.00                    1       1
      80.00                    1       1
      79.00                    1       1
2     99.00                    1       1
...                          ...     ...
      39.00                    3       3
0     54.00                    3       3
2     74.00                    3       3
0     51.00                    3       3
2     65.00                    4       4

[232 rows x 2 columns]

In [54]:
nested_df

,End Year,Level,Reported Spending per student,Money Difference per school,Proficiency Rate,Predicted Classification,Rating
0,"2,022.00",1,"9,967.50","-25,037.50",54.00,2.00,3.00
1,"2,022.00",0,"13,906.25","-92,312.50",36.00,2.00,3.00
2,"2,022.00",0,"9,155.00","-75,516.67",76.00,0.00,1.00
3,"2,019.00",1,"13,230.00","38,400.00",80.00,0.00,1.00
4,"2,022.00",0,"11,515.00","5,750.00",61.00,2.00,2.00
...,...,...,...,...,...,...,...
2480,"2,019.00",1,"11,160.00","-4,200.00",67.00,2.00,3.00
2481,"2,022.00",1,"10,778.75","-7,300.00",54.00,2.00,3.00
2482,"2,019.00",0,"11,200.00","25,200.00",75.00,0.00,2.00
2483,"2,019.00",0,"11,880.00","-76,100.00",91.00,0.00,0.00


In [55]:
# Determine whether the data needs to be balanced. 
# The high values 0 & 1 are 0.3, medium 0.33, and the low values .037

nested_df['Rating'].value_counts(normalize=True)

2.00   0.33
3.00   0.30
1.00   0.21
0.00   0.08
4.00   0.08
Name: Rating, dtype: float64

In [56]:
nested_df['End Year'].value_counts() 

2,019.00    1244
2,022.00    1241
Name: End Year, dtype: int64

In [57]:
nested_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   End Year                       2485 non-null   float64
 1   Level                          2485 non-null   int64  
 2   Reported Spending per student  2485 non-null   float64
 3   Money Difference per school    2485 non-null   float64
 4   Proficiency Rate               2485 non-null   float64
 5   Predicted Classification       2485 non-null   float64
 6   Rating                         2485 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 136.0 KB


In [58]:
nested_df

,End Year,Level,Reported Spending per student,Money Difference per school,Proficiency Rate,Predicted Classification,Rating
0,"2,022.00",1,"9,967.50","-25,037.50",54.00,2.00,3.00
1,"2,022.00",0,"13,906.25","-92,312.50",36.00,2.00,3.00
2,"2,022.00",0,"9,155.00","-75,516.67",76.00,0.00,1.00
3,"2,019.00",1,"13,230.00","38,400.00",80.00,0.00,1.00
4,"2,022.00",0,"11,515.00","5,750.00",61.00,2.00,2.00
...,...,...,...,...,...,...,...
2480,"2,019.00",1,"11,160.00","-4,200.00",67.00,2.00,3.00
2481,"2,022.00",1,"10,778.75","-7,300.00",54.00,2.00,3.00
2482,"2,019.00",0,"11,200.00","25,200.00",75.00,0.00,2.00
2483,"2,019.00",0,"11,880.00","-76,100.00",91.00,0.00,0.00


In [59]:
nested_inputs = nested_df.values[:,:-1]
nested_targets = nested_df.values[:,-1]

In [60]:
x_train, x_set, y_train, y_set = train_test_split(nested_inputs, nested_targets, test_size=0.2, random_state=3)
x_valid, x_test, y_valid, y_test = train_test_split(x_set, y_set, test_size=0.5, random_state=16)

In [61]:
# Scale data
nested_model_scaler = StandardScaler()

# Calculate and store the mean and sd
nested_model_scaler.fit(x_train)

# Apply the scaler
x_train = nested_model_scaler.transform(x_train)
x_valid = nested_model_scaler.transform(x_valid)
x_test = nested_model_scaler.transform(x_test)

In [62]:
x_train.shape

(1988, 6)

In [63]:
y_train.shape

(1988,)

## Model

In [64]:
# Create the model
# Ran different rounds of the below model.  Some of the options I put in comments to the right.

input_size = 6
output_size = 5
hidden_layer_size = 15 # also tried 20, 50, 10

nested_model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax') 
    ])

nested_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# tanh, tanh, softmax - val accuracy .922
# relu, relu, softmax - val accuracy .9366
# relu, tanh, softmax - val accuracy .9317
# relu, linear, softmax - val accuracy .9415
# linear, softmax - val accuracy .8048
# relu, softmax - val accuracy .8098

In [65]:
# Fit the model

batch_size = 50  # alternatives tried 200(.89), 20(.95), 50(.94), 40(.94), 100(.92)
max_epochs = 100 

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2) 

nested_model.fit(x_train, y_train, 
          batch_size=batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (x_valid, y_valid),
          verbose =2)


Epoch 1/100
40/40 - 1s - loss: 1.6451 - accuracy: 0.2550 - val_loss: 1.5717 - val_accuracy: 0.3548 - 1s/epoch - 37ms/step
Epoch 2/100
40/40 - 0s - loss: 1.4781 - accuracy: 0.4160 - val_loss: 1.4390 - val_accuracy: 0.4032 - 175ms/epoch - 4ms/step
Epoch 3/100
40/40 - 0s - loss: 1.3712 - accuracy: 0.4376 - val_loss: 1.3246 - val_accuracy: 0.4274 - 135ms/epoch - 3ms/step
Epoch 4/100
40/40 - 0s - loss: 1.2665 - accuracy: 0.4864 - val_loss: 1.2093 - val_accuracy: 0.5524 - 121ms/epoch - 3ms/step
Epoch 5/100
40/40 - 0s - loss: 1.1597 - accuracy: 0.5845 - val_loss: 1.0897 - val_accuracy: 0.6250 - 136ms/epoch - 3ms/step
Epoch 6/100
40/40 - 0s - loss: 1.0528 - accuracy: 0.6318 - val_loss: 0.9813 - val_accuracy: 0.6573 - 111ms/epoch - 3ms/step
Epoch 7/100
40/40 - 0s - loss: 0.9525 - accuracy: 0.6574 - val_loss: 0.8795 - val_accuracy: 0.6653 - 133ms/epoch - 3ms/step
Epoch 8/100
40/40 - 0s - loss: 0.8617 - accuracy: 0.6806 - val_loss: 0.7917 - val_accuracy: 0.7137 - 109ms/epoch - 3ms/step
Epoch 9/10

In [66]:
# test the model
nested_test_loss, nested_test_accuracy = nested_model.evaluate(x_test, y_test)


# Print Pretty
print('\nNested Test loss: {0:.2f}. Nested Test accuracy: {1:.2f}%'.format(nested_test_loss, nested_test_accuracy*100.))

8/8 [==============================] - 0s 3ms/step - loss: 0.2403 - accuracy: 0.9357

Nested Test loss: 0.24. Nested Test accuracy: 93.57%


In [67]:
# Save model
# Test accuracy for the saved model is 95.58%
#tf.keras.models.save_model(nested_model, '/content/drive/MyDrive/Colab Data/KY EPSB/nested_model', include_optimizer=True)

## Nested Predict

In [68]:
nested_inputs = predict_df.loc[:,['End Year', 'Level', 'Reported Spending per student', 'Money Difference per school',
                                  'Proficiency Rate',  'Predicted Classification']]

In [69]:
# Apply the scaler   
nested_inputs = nested_model_scaler.transform(nested_inputs.values)

# Predict Ky Rating
predict_rating = nested_model.predict(nested_inputs)

342/342 [==============================] - 1s 2ms/step


In [70]:
predict_rating.shape

(10922, 5)

In [71]:
# Convert to a df
predict_rating = pd.DataFrame(predict_rating, columns=['*Very High', '*High', '*Medium', '*Low', '*Very Low'])
# The * stands for predicted data

In [72]:
# Select the column with the highest percentage
predict_rating = predict_rating.idxmax(axis=1)

# Create the Predicted Rating column
predict_df['Predicted Rating'] = predict_rating

In [73]:
# Reformat into categories

predict_df['Classification'].replace([0, 1, 2], ['Distinguished', 'Proficient', 'Needs Improvement'], inplace=True)
predict_df['Predicted Classification'].replace([0, 1, 2], ['*Distinguished', '*Proficient', '*Needs Improvement'], inplace=True)
# The * stands for predicted data. 
predict_df['Level'].replace([0, 1, 2, 3], ['ES', 'MS', 'HS', np.NaN], inplace=True)

In [74]:
# Fill in the values with the predicted classification /rating
predict_df['Classification'].fillna(predict_df['Predicted Classification'], inplace = True)
predict_df['Rating'].fillna(predict_df['Predicted Rating'], inplace = True)

# Drop the Predicted columns
predict_df.drop(['Predicted Classification', 'Predicted Rating'], axis=1, inplace=True)

In [75]:
predict_df['Classification'].value_counts(dropna=False)

Needs Improvement     3265
*Needs Improvement    2517
*Distinguished        1624
Distinguished         1607
Proficient            1505
*Proficient            375
NaN                     29
Name: Classification, dtype: int64

In [76]:
predict_df['Rating'].value_counts(normalize=True, dropna=False)

*Medium      0.37
*Low         0.21
*Very Low    0.15
Medium       0.07
Low          0.07
High         0.05
*High        0.03
Very High    0.02
Very Low     0.02
*Very High   0.01
NaN          0.00
Name: Rating, dtype: float64

In [77]:
predict_df

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
0,"2,012.00",1,Adair County,1010,Adair County High School,HS,55.00,Needs Improvement,*Low,"6,460.00","-36,400.00",-50.00,"6,405.00"
1,"2,012.00",1,Adair County,1014,Adair County Middle School,MS,48.00,Needs Improvement,*Very Low,"6,720.00","-55,700.00",-140.00,"6,575.00"
2,"2,012.00",1,Adair County,1016,Adair County Elementary School,ES,63.00,Proficient,*Low,"7,260.00","-33,800.00",-60.00,"7,204.00"
3,"2,012.00",5,Allen County,5010,Allen County Primary Center,ES,50.00,Needs Improvement,*Very Low,"7,930.00","190,500.00",210.00,"8,137.00"
4,"2,012.00",5,Allen County,5015,James E Bazzell Middle School,MS,57.00,Needs Improvement,*Low,"7,640.00","78,600.00",180.00,"7,822.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,"2,022.00",601,Woodford County,601050,Southside Elementary School,ES,75.00,*Distinguished,High,"10,725.00","70,762.50",136.25,"10,861.75"
10918,"2,022.00",601,Woodford County,601075,Simmons Elementary School,ES,70.00,*Needs Improvement,Medium,"12,065.00","-14,650.00",-30.00,"12,033.38"
10919,"2,022.00",601,Woodford County,601084,Woodford County High School,HS,68.00,*Needs Improvement,High,"9,492.50","28,962.50",23.75,"9,515.38"
10920,"2,022.00",601,Woodford County,601090,Huntertown Elementary School,ES,77.00,*Distinguished,High,"10,847.50","115,625.00",247.50,"11,094.25"


In [78]:
predict_df.to_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/predict_df.csv', index = False)

# Most Impacted schools

In [82]:
# This makes us start with the predict_df. 

published_ky_df = predict_df.copy()
published_ky_df.reset_index(drop=True, inplace=True)

In [83]:
published_ky_df.index

RangeIndex(start=0, stop=10922, step=1)

In [84]:
published_ky_df.describe()

,End Year,District Code,School Code,Proficiency Rate,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
count,"10,922.00","10,922.00","10,922.00","10,922.00","10,876.00","10,876.00","10,876.00","10,876.00"
mean,"2,016.29",286.65,"286,761.55",65.31,"10,545.83","23,018.70",41.67,"10,563.31"
std,2.99,163.98,"164,002.16",12.69,"3,323.51","116,519.98",194.25,"3,354.19"
min,"2,012.00",1.00,"1,010.00",10.00,"2,340.00","-666,200.00","-1,400.00",48.00
25%,"2,014.00",165.00,"165,018.00",57.00,"8,299.38","-15,800.00",-35.00,"8,303.00"
50%,"2,016.00",275.00,"275,106.00",66.00,"9,998.57","22,800.00",50.00,"10,032.00"
75%,"2,018.00",435.00,"435,030.00",74.00,"12,080.00","59,403.12",130.00,"12,156.75"
max,"2,022.00",606.00,"606,460.00",111.00,"69,010.00","1,497,800.00","3,520.00","69,074.00"


## Top/Bottom Percentile

In [85]:
def select_heavily_impacted_schools(published_ky_df, selected_percentile):
  numeric_columns = ['Money Difference per school', 'Estimated Spending per student', 'Proficiency Rate']
  quantiles_by_year = published_ky_df.groupby('End Year')[numeric_columns].quantile([selected_percentile, 1 - selected_percentile])

  heavily_impacted_schools = pd.DataFrame()

  for year in published_ky_df['End Year'].unique():
      year_data = published_ky_df[published_ky_df['End Year'] == year]

      highest_pctl = quantiles_by_year.loc[year, selected_percentile]
      lowest_pctl = quantiles_by_year.loc[year, 1 - selected_percentile]

      # Create boolean masks for each condition
      high_money_diff = year_data['Money Difference per school'] > highest_pctl['Money Difference per school']
      low_money_diff = year_data['Money Difference per school'] < lowest_pctl['Money Difference per school']
      high_spending = year_data['Estimated Spending per student'] > highest_pctl['Estimated Spending per student']
      low_spending = year_data['Estimated Spending per student'] < lowest_pctl['Estimated Spending per student']
      high_proficiency = year_data['Proficiency Rate'] > highest_pctl['Proficiency Rate']
      low_proficiency = year_data['Proficiency Rate'] < lowest_pctl['Proficiency Rate']

      # Use boolean indexing to select rows that meet any of the conditions
      year_selected_rows = year_data[high_money_diff | low_money_diff | high_spending | low_spending | high_proficiency | low_proficiency]

      # Add the selected rows to the overall DataFrame
      heavily_impacted_schools = pd.concat([heavily_impacted_schools, year_selected_rows])

  # Remove duplicates and reset index
  heavily_impacted_schools = heavily_impacted_schools.drop_duplicates().reset_index(drop=True)

  return heavily_impacted_schools



In [86]:
# Select value between 0-100. The selected value will be changed to a percent.  Then used to pull the data below. 

select_percent = 5
selected_percentile = (100 - select_percent) / 100

# Current parameters make this set to the top and bottom 5%. 

highly_impacted_schools = select_heavily_impacted_schools(published_ky_df, selected_percentile)

In [87]:
published_ky_df[published_ky_df['End Year']==2018].quantile(q=selected_percentile)

End Year                           2,018.00
District Code                        567.40
School Code                      567,418.20
Proficiency Rate                      91.00
Reported Spending per student     21,457.50
Money Difference per school      175,800.00
Money Difference per student         330.00
Estimated Spending per student    21,529.00
Name: 0.95, dtype: float64

In [88]:
select_heavily_impacted_schools(published_ky_df, 0.99)

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
0,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,ES,71.00,Distinguished,*Medium,"19,951.25","123,650.00",335.00,"20,284.38"
1,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,MS,92.00,Distinguished,*Medium,"19,850.00","50,100.00",140.00,"19,984.00"
2,"2,012.00",15,Ballard County,15005,Ballard County Middle School,MS,63.00,Proficient,*Medium,"4,760.00","-32,500.00",-100.00,"4,655.00"
3,"2,012.00",16,Barbourville Independent,16011,Barbourville City School,HS,57.00,Needs Improvement,*Medium,"4,670.00","15,700.00",20.00,"4,694.00"
4,"2,012.00",26,Beechwood Independent,26020,Beechwood High School,HS,84.00,Distinguished,*Medium,"6,910.00","-1,100.00",-0.00,"6,906.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,"2,022.00",501,Pulaski County,501395,Northern Middle School,MS,69.00,*Proficient,High,"7,230.00","-142,800.00",-165.00,"7,065.62"
590,"2,022.00",585,Webster County,585050,Dixon Elementary School,ES,60.00,*Needs Improvement,Medium,"6,467.50","38,687.50",103.75,"6,570.88"
591,"2,022.00",585,Webster County,585090,Sebree Elementary School,ES,50.00,*Needs Improvement,Low,"7,003.75","34,612.50",93.75,"7,099.50"
592,"2,022.00",585,Webster County,585125,Webster County Middle School,MS,54.00,*Needs Improvement,Low,"6,266.00","33,400.00",98.00,"6,362.00"


In [89]:
highly_impacted_schools

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
0,"2,012.00",5,Allen County,5010,Allen County Primary Center,ES,50.00,Needs Improvement,*Very Low,"7,930.00","190,500.00",210.00,"8,137.00"
1,"2,012.00",5,Allen County,5020,Allen County-Scottsville High School,HS,51.00,Needs Improvement,*Low,"8,500.00","157,000.00",180.00,"8,685.00"
2,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,ES,71.00,Distinguished,*Medium,"19,951.25","123,650.00",335.00,"20,284.38"
3,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,MS,92.00,Distinguished,*Medium,"19,850.00","50,100.00",140.00,"19,984.00"
4,"2,012.00",11,Anderson County,11022,Robert B. Turner Elementary School,ES,67.00,Proficient,*Medium,"5,550.00","-72,000.00",-150.00,"5,399.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,"2,022.00",591,Whitley County,591062,Whitley County East Elementary School,ES,84.00,Distinguished,Very High,"12,712.50","-40,712.50",-168.75,"12,542.62"
2710,"2,022.00",595,Wolfe County,595205,Red River Valley Elementary School,ES,30.00,Needs Improvement,Very Low,"15,167.50","-13,175.00",-83.75,"15,086.00"
2711,"2,022.00",595,Wolfe County,595220,Rogers Elementary School,ES,53.00,*Needs Improvement,Low,"14,825.00","-7,200.00",-48.75,"14,778.25"
2712,"2,022.00",595,Wolfe County,595315,Wolfe County Middle School,MS,46.00,*Needs Improvement,Low,"14,397.50","-9,400.00",-47.50,"14,350.12"


In [90]:
# Without including the Proficeny Rate high and low, there were 1941 records, and 701 schools.
# Including the proficency rate raises the number of records to 2483 from 817 schools.

highly_impacted_schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2714 entries, 0 to 2713
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   End Year                        2714 non-null   float64
 1   District Code                   2714 non-null   int64  
 2   District                        2714 non-null   object 
 3   School Code                     2714 non-null   int64  
 4   School                          2714 non-null   object 
 5   Level                           2714 non-null   object 
 6   Proficiency Rate                2714 non-null   float64
 7   Classification                  2713 non-null   object 
 8   Rating                          2708 non-null   object 
 9   Reported Spending per student   2706 non-null   float64
 10  Money Difference per school     2706 non-null   float64
 11  Money Difference per student    2706 non-null   float64
 12  Estimated Spending per student  27

In [91]:
highly_impacted_schools.nunique()

End Year                             9
District Code                      160
District                           160
School Code                        763
School                             754
Level                                3
Proficiency Rate                    99
Classification                       6
Rating                              10
Reported Spending per student     1459
Money Difference per school       2073
Money Difference per student       417
Estimated Spending per student    2317
dtype: int64

In [92]:
highly_impacted_schools.sort_values(by=['Money Difference per school'], inplace=True)

## Multiple Years Impacted - Detailed

In [93]:
# If school code frequency is >= 5, then they remain.  
# Then the rest of their 10 year data is pulled. 

In [94]:
id_freq_schools = highly_impacted_schools.groupby('School Code').count()

In [95]:
id_freq_schools.describe()

,End Year,District Code,District,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
count,763.00,763.00,763.00,763.00,763.00,763.00,763.00,763.00,763.00,763.00,763.00,763.00
mean,3.56,3.56,3.56,3.56,3.56,3.56,3.56,3.55,3.55,3.55,3.55,3.55
std,3.05,3.05,3.05,3.05,3.05,3.05,3.04,3.04,3.05,3.05,3.05,3.05
min,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00
75%,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
max,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00


In [96]:
# Select frequency that a school appears in either the top or bottom percentile of the highly impacted schools
# There should be a default selected. 

select_frequency = 5

# If school code frequency is >= 5 years, then they remain. Unless above variable is updated to a different number.   

for i in id_freq_schools.index:
  if id_freq_schools['End Year'][i] < select_frequency:
    id_freq_schools.drop([i], axis= 0, inplace=True)

freq_school_code = id_freq_schools.index.unique().sort_values()

In [97]:
id_freq_schools.sort_values(by='End Year')

,End Year,District Code,District,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
School Code,,,,,,,,,,,,
275116,5,5,5,5,5,5,5,5,5,5,5,5
585130,5,5,5,5,5,5,5,5,5,5,5,5
465014,5,5,5,5,5,5,5,5,5,5,5,5
567020,5,5,5,5,5,5,5,5,5,5,5,5
165025,5,5,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6010,18,18,18,18,18,18,18,18,18,18,18,18
275155,18,18,18,18,18,18,18,18,18,18,18,18
165018,18,18,18,18,18,18,18,18,18,18,18,18


In [98]:
id_freq_schools.describe()

,End Year,District Code,District,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
count,214.00,214.00,214.00,214.00,214.00,214.00,214.00,214.00,214.00,214.00,214.00,214.00
mean,7.54,7.54,7.54,7.54,7.54,7.54,7.53,7.51,7.50,7.50,7.50,7.50
std,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.92,2.93,2.93,2.93,2.93
min,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00,0.00,0.00,0.00,0.00
25%,6.00,6.00,6.00,6.00,6.00,6.00,6.00,5.25,5.25,5.25,5.25,5.25
50%,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00
75%,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00
max,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00


In [99]:
freq_school_code

Int64Index([  6010,  11022,  11025,  16011,  21027,  21050,  21150,  26010,
             26020,  31180,
            ...
            505010, 525014, 537010, 567020, 571002, 571230, 585050, 585125,
            585130, 591061],
           dtype='int64', name='School Code', length=214)

In [ ]:
# This is the detailed schools who are repeatedly in the top/bottom.
# Pulls all of their school data. 

ten_year_impacted = published_ky_df.iloc[0:0,:]

for code in freq_school_code:
  school_ten_year = published_ky_df[published_ky_df['School Code'] == code]
  ten_year_impacted = ten_year_impacted.append(school_ten_year)


In [101]:
ten_year_impacted.sort_values(by=['Money Difference per school'], inplace=True)

In [102]:
ten_year_impacted.tail(20)

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
6508,"2,017.00",275,Jefferson County,275018,Atherton High,HS,76.00,*Distinguished,*Medium,"11,930.00","832,300.00",580.00,"12,513.00"
7733,"2,018.00",275,Jefferson County,275018,Atherton High,HS,79.00,*Distinguished,*High,"13,970.00","850,400.00",600.00,"14,563.00"
8969,"2,019.00",275,Jefferson County,275007,Eastern High,HS,69.00,*Needs Improvement,High,"13,130.00","920,800.00",470.00,"13,593.00"
7726,"2,018.00",275,Jefferson County,275007,Eastern High,HS,74.00,*Proficient,*High,"13,140.00","959,000.00",480.00,"13,613.00"
2988,"2,014.00",275,Jefferson County,275200,duPont Manual High,HS,92.00,Distinguished,*High,"7,840.00","1,003,900.00",540.00,"8,385.00"
10320,"2,022.00",275,Jefferson County,275200,duPont Manual High,HS,99.00,Distinguished,Very High,"10,777.50","1,037,600.00",551.25,"11,327.38"
8988,"2,019.00",275,Jefferson County,275047,Louisville Male High,HS,78.00,Distinguished,Very High,"12,640.00","1,071,800.00",560.00,"13,196.00"
5391,"2,016.00",275,Jefferson County,275200,duPont Manual High,HS,91.00,Distinguished,*Very High,"11,740.00","1,258,300.00",670.00,"12,410.00"
6598,"2,017.00",275,Jefferson County,275200,duPont Manual High,HS,100.00,*Distinguished,*Very High,"12,380.00","1,265,600.00",670.00,"13,050.00"
7822,"2,018.00",275,Jefferson County,275200,duPont Manual High,HS,105.00,*Distinguished,*Very High,"13,460.00","1,304,000.00",680.00,"14,138.00"


In [103]:
ten_year_impacted.nunique()

End Year                             9
District Code                       57
District                            57
School Code                        214
School                             213
Level                                3
Proficiency Rate                    99
Classification                       6
Rating                              10
Reported Spending per student     1155
Money Difference per school       1645
Money Difference per student       348
Estimated Spending per student    1787
dtype: int64

In [104]:
dist_school_print = ten_year_impacted.loc[:,['District', 'School']]

dist_school_print.drop_duplicates(inplace=True)
dist_school_print.reset_index(drop=True, inplace=True)

In [105]:
dist_school_print

,District,School
0,Jefferson County,Frederick Law Olmsted Academy North
1,Jefferson County,Marion C. Moore School
2,Jefferson County,Valley High
3,Jefferson County,Western High
4,Jefferson County,Thomas Jefferson Middle
...,...,...
209,Jefferson County,St Matthews Elementary
210,Jefferson County,Bowen Elementary
211,Jefferson County,duPont Manual High
212,Jefferson County,Jefferson County Traditional Middle


In [106]:
dist_school_print[dist_school_print['District'].str.contains("Jefferson County")]

,District,School
0,Jefferson County,Frederick Law Olmsted Academy North
1,Jefferson County,Marion C. Moore School
2,Jefferson County,Valley High
3,Jefferson County,Western High
4,Jefferson County,Thomas Jefferson Middle
...,...,...
209,Jefferson County,St Matthews Elementary
210,Jefferson County,Bowen Elementary
211,Jefferson County,duPont Manual High
212,Jefferson County,Jefferson County Traditional Middle


In [107]:
ten_year_impacted.sort_values(by=['District', 'School'])

,End Year,District Code,District,School Code,School,Level,Proficiency Rate,Classification,Rating,Reported Spending per student,Money Difference per school,Money Difference per student,Estimated Spending per student
8,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,MS,92.00,Distinguished,*Medium,"19,850.00","50,100.00",140.00,"19,984.00"
3561,"2,015.00",6,Anchorage Independent,6010,Anchorage Independent Public School,MS,79.00,Distinguished,*Medium,"18,500.00","67,100.00",180.00,"18,681.00"
1188,"2,013.00",6,Anchorage Independent,6010,Anchorage Independent Public School,MS,79.00,Distinguished,*Medium,"18,280.00","115,800.00",310.00,"18,584.00"
7,"2,012.00",6,Anchorage Independent,6010,Anchorage Independent Public School,ES,71.00,Distinguished,*Medium,"19,951.25","123,650.00",335.00,"20,284.38"
9664,"2,022.00",6,Anchorage Independent,6010,Anchorage Independent Public School,ES,98.00,Distinguished,Very High,"19,951.25","123,650.00",335.00,"20,284.38"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,"2,014.00",591,Whitley County,591061,Whitley Central Intermediate School,ES,76.00,Distinguished,*Medium,"7,940.00","-166,200.00",-380.00,"7,561.00"
10898,"2,022.00",591,Whitley County,591061,Whitley Central Intermediate School,ES,79.00,*Distinguished,High,"9,951.25","-142,225.00",-316.25,"9,635.75"
7147,"2,017.00",591,Whitley County,591061,Whitley Central Intermediate School,ES,82.00,*Distinguished,*Medium,"10,800.00","-77,200.00",-170.00,"10,624.00"
9632,"2,019.00",591,Whitley County,591061,Whitley Central Intermediate School,ES,91.00,Distinguished,Very High,"11,880.00","-76,100.00",-170.00,"11,719.00"
